In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dotenv import load_dotenv

from computer_use_demo.tools import ComputerTool, ToolCollection

load_dotenv()

from computer_use_demo.loop import completion, PROVIDER_TO_DEFAULT_MODEL_NAME, APIProvider, SYSTEM_PROMPT
import os

from playwright.async_api import async_playwright

In [ ]:
from print_utils import print_conversation, noop

In [ ]:
messages = []

In [ ]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
page = await browser.new_page(viewport={"width": 1024, "height": 768})

In [ ]:
import base64

await page.goto("https://stackoverflow.com")
screenshot = await page.screenshot(timeout=0)
screenshot_base64 = base64.b64encode(screenshot).decode("utf-8")
prompt = {
    "role": "user",
    "content": [
        {
            "type": "text",
            "text": "Given the following page, perform all the necessary actions to dismiss/close any popups/dialogs you see. Do not take a screenshot, work with the provided page."
        },
        {
            "type": "image",
            "source": {
                "type": "base64",
                "media_type": "image/png",
                "data": screenshot_base64,
            },
        }
    ]
}
messages.append(prompt)

In [ ]:
# import base64
# 
# await page.goto("https://www.bidnetdirect.com/colorado/cityandcountyofdenverdepartmentofaviation")
# screenshot = await page.screenshot(timeout=0)
# screenshot_base64 = base64.b64encode(screenshot).decode("utf-8")
# prompt = {
#     "role": "user",
#     "content": [
#         {
#             "type": "text",
#             "text": "Given the following page, find the closed contracts. Do not ask for a screenshot, work with the provided screenshot."
#         },
#         {
#             "type": "image",
#             "source": {
#                 "type": "base64",
#                 "media_type": "image/png",
#                 "data": screenshot_base64,
#             },
#         }
#     ]
# }
# messages = [prompt]

In [ ]:
# import base64
# 
# await page.goto("https://smctd.bonfirehub.com/portal/?tab=openOpportunities")
# screenshot = await page.screenshot(timeout=0)
# screenshot_base64 = base64.b64encode(screenshot).decode("utf-8")
# prompt = {
#     "role": "user",
#     "content": [
#         {
#             "type": "text",
#             "text": "Given the following page, go to the details page of the first contract you see. Do not ask for a screenshot, work with the provided screenshot."
#         },
#         {
#             "type": "image",
#             "source": {
#                 "type": "base64",
#                 "media_type": "image/png",
#                 "data": screenshot_base64,
#             },
#         }
#     ]
# }
# messages = [prompt]

In [ ]:
tool_collection = ToolCollection(ComputerTool(page))
response = await completion(
    model=PROVIDER_TO_DEFAULT_MODEL_NAME[APIProvider.ANTHROPIC],
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
    provider=APIProvider.ANTHROPIC,
    messages=messages,
    api_response_callback=noop,
    tool_collection=tool_collection,
)
messages.append({
    "role": "assistant",
    "content": response,
})
print_conversation(messages)

In [ ]:
print_conversation(messages)

In [ ]:
from typing import Any, cast

tool_result_content = []
for content_block in response:
    if content_block["type"] == "tool_use":
        result = await tool_collection.run(
            name=content_block["name"],
            tool_input=cast(dict[str, Any], content_block["input"]),
        )
        tool_result_content.append(
            (result, content_block["id"])
        )
tool_result_content